# Numeric backend interface

## Example wrapping uncertainties : add the unit to `nominal_value` and `std_dev`

Numpy's integration is a great example of how physipy can wrap any kind of numerical values, but this integration is written in the source code of physipy so it's a bit cheating.  
Let's see how physipy deals with a non-integrated numerical-like package : uncertainties. By "non-integrated" I mean that no source code of physipy makes any explicit reference to uncertainties, so we only rely on the general wrapping interface of physipy.

In [3]:
from physipy import m, s, K, Quantity, Dimension
import uncertainties.umath as um
import uncertainties as u

# create a pure uncertainties instance
x = u.ufloat(0.20, 0.01)  # x = 0.20+/-0.01
print(x)
print(x**2)
print(type(x))

0.200+/-0.010
0.040+/-0.004
<class 'uncertainties.core.Variable'>


If we create a quantity version by mulitplying by 1 meter, the returned value is of Quantity type:

In [4]:
# now let's create a quanity version of x
xq = x*m
print(xq)                    # a Quantity
print(xq**2, type(xq**2))    # a Quantity
print(xq+2*m, type(xq+2*m))  # a Quantity
print(xq.value)              # an uncertainties value
print(m*x == x*m)            # True

0.200+/-0.010 m
0.040+/-0.004 m**2 <class 'physipy.quantity.quantity.Quantity'>
2.200+/-0.010 m <class 'physipy.quantity.quantity.Quantity'>
0.200+/-0.010
True


That's a pretty neat result __that didn't need any additional code__.  
Now going a bit further, uncertainties instance have a `nominal_value` and `std_dev` attributes.

In [5]:
# Creation must be done this way and not by "x*m" because "x*m" 
# will multiply the uncerainties variable by 1, and turn it into a
# AffineScalarFunc instance, which is not hashable and breaks my 
# register_property_backend that relies on dict lookup
#x = u.ufloat(0.20, 0.01)  # x = 0.20+/-0.01
xq = Quantity(x, Dimension("m")) # xq = x *m

In [6]:
print(x.nominal_value)
print(x.std_dev)

0.2
0.01


In physipy, if an attribute doesn't exist in the quantity instance, the lookup falls back on the backend value, ie on the uncertainties variable, so by default we get the same result on `xq` (note that we don't get auto-completion either for the same reason):

In [7]:
print(xq.nominal_value) # 0.2
print(xq.std_dev)       # 0.1

0.2 m
0.01 m


It would be great that `xq.nominal_value` actually prints `0.2 m`, not loosing the unit and making it explicit that the nominal value is actually 0.2 meters. To do that, we can add a property back to specify what we want `xq.nominal_value` to return : a property backend is a dictionnary with key the name of the attribute, and as value the corresponding method to get the wanted result.

For the nominal_value and standard deviation, we just want to add back the unit and make the variable a Quantity, so we multiply by the corresponding SI unit:

In [8]:
type(xq.value)

uncertainties.core.Variable

In [9]:
import uncertainties as uc
from physipy.quantity.quantity import register_property_backend

uncertainties_property_backend_interface = {
    # res is the backend result of the attribute lookup, and q the wrapping quantity
    "nominal_value":lambda q, res: q._SI_unitary_quantity*res,
    "std_dev":lambda q, res: q._SI_unitary_quantity*res,
    "n":lambda q, res: q._SI_unitary_quantity*res,
    "s":lambda q, res: q._SI_unitary_quantity*res,
}

print("Registering uncertainties")
register_property_backend(uc.core.Variable, 
                         uncertainties_property_backend_interface)

Registering uncertainties


With this property back interface registered we get the desired result for `print(xq.nominal_value)`:

In [10]:
print(type(xq.value))
print(xq.nominal_value) # 0.2 m, instead of just 0.2 previously
print(xq.std_dev)       # 0.1 m, instead of just 0.1 previously

<class 'uncertainties.core.Variable'>
0.2 m
0.01 m


In [11]:
yq = 2*xq 

In [12]:
type(list(yq.derivatives.keys())[0])

uncertainties.core.Variable

In [19]:
from physipy.math import decorator_angle_or_dimless_to_dimless
from physipy import rad

In [21]:
sin = decorator_angle_or_dimless_to_dimless(um.sin)
print(sin(x*rad))
print(um.sin(x))

0.199+/-0.010
0.199+/-0.010


## Why the duck type approach doesn't work

Another approach to do this would be to create a new class like this

In [8]:
from physipy import m, s, K, Quantity, Dimension
import uncertainties as u

# create a pure uncertainties instance
x = u.ufloat(0.20, 0.01)  # x = 0.20+/-0.01

class UWrappedQuantity(Quantity):
    @property
    def nominal_value(self):
        return self.value.nominal_value * self._SI_unitary_quantity
    @property
    def std_dev(self):
        return self.value.std_dev * self._SI_unitary_quantity

xq2 = UWrappedQuantity(x, Dimension("m"))
#print(xq2)
print(type(xq2), xq2, xq2.nominal_value)

<class '__main__.UWrappedQuantity'> 0.200+/-0.010 m 0.2 m


But with this definition, newly created instance will be of type `Quantity`, not `UWrappedQuantity`, loosing again the unit on `nominal_value`

In [9]:
print(xq2+2*m, type(xq2+2*m), (xq2+2*m).value, (xq2+2*m)._SI_unitary_quantity)
print((xq2+2*m).nominal_value)

2.200+/-0.010 m <class 'physipy.quantity.quantity.Quantity'> 2.200+/-0.010 1 m
2.2


## Interface

The simplest way to use physipy with specific objects, like fractions or your own class is to create quantities that wrap your value : here `Quantity` wraps the custom value as its `value` attribute.

A simple example : 

In [10]:
import fractions
from physipy import Quantity, Dimension, m

In [11]:
length = Quantity(fractions.Fraction(3, 26), Dimension("m"))
print(length)

3/26 m


Then when doing calculation, physipy deals with everything for you :

In [12]:
print(length + 2*m)
print(length**2)
print(length.dimension)
print(length.is_mass())
print(length.sum())

55/26 m
9/676 m**2
L
False
3/26 m


# More complex objects

Now say we want to customize the basic Fraction object, by overloading its str method : 

In [13]:
class MyFraction(fractions.Fraction):
    def __str__(self):
        return f"[[[{self.numerator}/{self.denominator}]]]"

In [14]:
my_length = MyFraction(3, 26)
print(my_length)
print(my_length*my_length)
print(Quantity(MyFraction(3, 26), Dimension("L")))
print(MyFraction(3, 26)*m)

[[[3/26]]]
9/676
[[[3/26]]] m
3/26 m


Notice that we lost the custom str : that's because the value is not a `MyFraction` instance but `fractions.Fraction` : it was lost in the multiplication process. Indeed, since `my_length*m` falls back on the `__mul__` of Quantity, that uses the `__mul__` method of the instance, which is `fractions.Fraction.__mul__`, which returns a `fractions.Fraction` instance, not a `MyFraction` instance.

So we would like physipy to return a `MyFraction` when computing multiplication with a Quantity objet.

Now we can't expect each user to rewrite its custom Fraction class to be compatible with Quantity, so we do the opposite : Quantity will wrap the custom class with an interface class.

In [15]:
class QuantityWrappedMyFraction(Quantity):
    def __str__(self):
        str_myfraction = str(self.value)
        return "QuantityWrappedMyFraction : " + str_myfraction + "-"+self.dimension.str_SI_unit()

from physipy.quantity.quantity import register_value_backend
register_value_backend(
    # here we use the class of the base value
    fractions.Fraction, 
    # here the rewritten version
    QuantityWrappedMyFraction)

In [16]:
print(my_length)
print(my_length*my_length)
print(Quantity(MyFraction(3, 26), Dimension("L")))
print(MyFraction(3, 26)*m)

[[[3/26]]]
9/676
[[[3/26]]] m
QuantityWrappedMyFraction : 3/26-m


# Requirements for easy backend supports

 - The class must be hashable : for the dict lookup on type, we need the class as key, so they need to be hashable